## Imports

In [ ]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

## Prepare dataset

In [7]:

path_to_data = Path('../data')
data = pd.read_csv(path_to_data / '2018_10_18_trump.csv', header=None, nrows=1000)
cards = [
    # Diamonds
    'DA', 'DK', 'DQ', 'DJ', 'D10', 'D9', 'D8', 'D7', 'D6',
    # Hearts
    'HA', 'HK', 'HQ', 'HJ', 'H10', 'H9', 'H8', 'H7', 'H6',
    # Spades
    'SA', 'SK', 'SQ', 'SJ', 'S10', 'S9', 'S8', 'S7', 'S6',
    # Clubs
    'CA', 'CK', 'CQ', 'CJ', 'C10', 'C9', 'C8', 'C7', 'C6'
]
forehand = ['FH']

user = ['user']
trump = ['trump']

data.columns = cards + forehand + user + trump
data.drop('user', axis='columns', inplace=True)

data['diamonds'] = data[['DA', 'DK', 'DQ', 'DJ', 'D10', 'D9', 'D8', 'D7', 'D6']].sum(axis=1)
data['hearts'] = data[['HA', 'HK', 'HQ', 'HJ', 'H10', 'H9', 'H8', 'H7', 'H6']].sum(axis=1)
data['spades'] = data[['SA', 'SK', 'SQ', 'SJ', 'S10', 'S9', 'S8', 'S7', 'S6']].sum(axis=1)
data['clubs'] = data[['CA', 'CK', 'CQ', 'CJ', 'C10', 'C9', 'C8', 'C7', 'C6']].sum(axis=1)

data['total_cards'] = data[cards].sum(axis=1)

data = pd.get_dummies(data, columns=['trump'], prefix='trump')
data.drop(columns=cards, inplace=True)

Epoch 1/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7613 - loss: 0.5718 - val_accuracy: 0.7250 - val_loss: 0.5834
Epoch 2/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7625 - loss: 0.5254 - val_accuracy: 0.7250 - val_loss: 0.5212
Epoch 3/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7650 - loss: 0.4913 - val_accuracy: 0.7500 - val_loss: 0.4929
Epoch 4/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7725 - loss: 0.4752 - val_accuracy: 0.7750 - val_loss: 0.4779
Epoch 5/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7725 - loss: 0.4595 - val_accuracy: 0.7850 - val_loss: 0.4634
Epoch 6/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7700 - loss: 0.4404 - val_accuracy: 0.7400 - val_loss: 0.4606
Epoch 7/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7688 - loss: 0.4282 - val_accuracy: 0.7650 - val_loss: 0.4343
Epoch 8/32
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7763 - loss: 0.4145 - val_accuracy: 0.7450 - val_loss:

## Model

In [ ]:
X = data.drop(columns=['FH']).values
y = data['FH'].values
X = X.astype('float32')
y = y.astype('float32')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=32, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

model.save('trump_model.keras')

# How to use

In [8]:
from tensorflow.keras.models import load_model

model = load_model('../trump_prediction/trump_model.keras')

hand = "placeholder for hand"
predictions = model.predict(hand)

KeyError: 'pop from an empty set'